# Data processing with Pandas and Dask









**Ariel Rossanigo**


### Who I am?

* Ariel Rossanigo
* Artificial Intelligence teacher at UCSE-DAR
* Developer, Data Scientist

### Goals

* Pandas intro
* Dask intro 


https://pandas.pydata.org/pandas-docs/stable/10min.html


### In order to run this notebook...

#### 1. Clone talks repository and move to this talk folder

    git clone git@github.com:arielrossanigo/data_processing_with_pandas_and_dask.git
    cd data_processing_with_pandas_and_dask

#### 2. Install requirements

    pip install -r requirements.txt

#### 3. Open this notebook

    jupyter notebook data_processing_with_pandas_and_dask.ipynb
  

#### 4. Run the next cell

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import output_notebook
output_notebook()
print("It works!")

### Pandas (Python Data Analysis Library)

* The *de facto* tool to work with data in Python
* It's made over **numpy** 
* It have high level data structures

### Basic structures

* Serie: one dimensional labeled array

In [ ]:
array = np.random.randn(5)
s = pd.Series(array, index=['a', 'b', 'c', 'd', 'e'])
s 

#### Data alignment is intrinsic

In [ ]:
array[1:] + array[:-1]

In [ ]:
s[1:] + s[:-1]

* Dataframe: 2-dimensional labeled data structure with columns of potentially different types

In [ ]:
beatles = pd.DataFrame({    
    'name': ['john', 'paul', 'george', 'ringo'],
    'year': [1940, 1942, 1943, 1940]
})
beatles

### Reading data into dataframes

The pandas I/O API is a set of top level reader and writer functions that supports various formats (CSV, Excel, HDF5, pickle, SQL and more)

http://pandas.pydata.org/pandas-docs/stable/io.html

#### The example data...

We'll have 2 sets of data:

* dropouts: accounts that dropped out from our company
* incomes: imcome of people related to the company accross the time

In [ ]:
dropouts = pd.read_csv('dropouts.csv.gz', 
                       parse_dates=['dropout_date'], 
                       index_col='account_id')

dropouts.head()

In [ ]:
incomes = pd.read_csv('incomes.csv.gz', 
                      parse_dates=['valid_from'])
incomes.info(memory_usage='deep')

### Data selection

* By label: **loc**, **at**

The dropout motive of the account_id 55174276247

In [ ]:
dropouts.at[55174276247, 'dropout_motive']

* By position: **iloc**, **iat**

The third row

In [ ]:
dropouts.iloc[2]

* Boolean indexing

Dropouts of 2018

In [ ]:
dropouts.loc[dropouts.dropout_date.dt.year == 2018].head()

### Column setting

In [ ]:
dropouts['year'] = dropouts.dropout_date.dt.year
dropouts.head()

### Operations

In [ ]:
(incomes.income + incomes.income).head()

In [ ]:
incomes.income.mean(), incomes.income.median()

In [ ]:
# How many dropouts per motive
dropouts.dropout_motive.value_counts().head(5)

In [ ]:
# pivot tables 
dropouts.pivot_table(index='dropout_motive', 
                     columns='year', 
                     aggfunc='count', 
                     margins=True)

In [ ]:
# ranges
bins = [0, 30_000, 50_000, 1e15]
incomes['range'] = pd.cut(incomes.income, bins)
display(incomes.range.value_counts())
del incomes['range']

### Missing data

In [ ]:
m = incomes[incomes.income.isnull()]
m.head()

In [ ]:
m.fillna(0).head()

In [ ]:
sa = incomes[
    incomes.account_id.isin([71545775181, 79712936137])
].copy()
sa['no_na'] = sa.income.fillna(method='bfill')
sa

### Applying functions  

In [ ]:
sa.income.apply(lambda x: '{:,.2f} $'.format(x)).head(3)

### Grouping data

Involves at least one of:

* **Splitting** data into groups based on some criteria
* **Applying** a function to each group independently
 * Aggregation
 * Transformation
 * Filtration
* **Combining** the results into a data structure

http://pandas.pydata.org/pandas-docs/stable/groupby.html

In [ ]:
# fixing the fillna problem with bfill
sa['no_na'] = sa.income.fillna(method='bfill')
sa['no_na_fixed'] = (sa.groupby('account_id')
                     .income
                     .fillna(method='bfill'))
sa

Let's suppose that we want to calculate the income of an account at the dropout moment...

In [ ]:
# lets create a valid_to column in order to have a simpler way of filter the income for an specific date
f_incomes = incomes[incomes.income.notnull()].copy()
f_incomes = f_incomes.sort_values(
    by=['account_id', 'employer', 'valid_from']
)
f_incomes['valid_to'] = f_incomes.groupby(
    ['account_id', 'employer']
).valid_from.shift(-1)
f_incomes.valid_to = f_incomes.valid_to.fillna(
    pd.to_datetime('21000101')
)

In [ ]:
f_incomes[f_incomes.account_id==2376461]

### Combining data
#### Concat

In [ ]:
d_2018 = dropouts[dropouts.year == 2018].sample(5)
d_2017 = dropouts[dropouts.year == 2017].sample(5)
sample_dropouts = pd.concat([d_2017, d_2018], axis='index')
sample_dropouts

### Combining data
#### Merge

In [ ]:
merged = dropouts.merge(f_incomes, 
                        left_index=True, 
                        right_on='account_id')
merged.head()

In [ ]:
merged = merged[(merged.valid_from <= merged.dropout_date) & 
                (merged.dropout_date < merged.valid_to)]
merged = merged.groupby(
    ['account_id', 'dropout_date', 'dropout_motive']
).income.sum().reset_index()
merged[merged.account_id==41608245309].head()

### Some charts

In [ ]:
(merged.income
 .clip_upper(merged.income.quantile(0.98))
 .hist(bins=20));

In [ ]:
merged['clipped_income'] = merged.income.clip_upper(merged.income.quantile(0.98))
ax = merged.boxplot(by='dropout_motive', 
                    column='clipped_income', 
                    figsize=(10, 3))

ax.set_xticklabels([x.get_text()[:20] for x in ax.get_xticklabels()], rotation=25, ha='right')
ax.set_title('')
del merged['clipped_income']

### Dask

<img src="imgs/collections-schedulers.png" width="600" align="middle">


### Dask

#### Why?

* Familiar API
* Single Computer & Cluster
* Integrates with the Python ecosystem
* Supports complex applications


### Dask Dataframe

<div style="float: left; margin: 30px;"><img src="imgs/dask-dataframe.svg" width="300" align="middle"></div>


**Common uses**

* Dataset doesn't fit in memory
* Accelerating long computations by using many cores
* Distribute computing with standard Pandas operations

**Anti uses**

* Data fits in memory
* Data is not tabular (use some other collection)
* Non standar operations are needed (use delayed)

In [ ]:
# let's improve previous calculation
import dask.dataframe as dd
import dask.diagnostics
from dask.diagnostics import (Profiler, ResourceProfiler, 
                              CacheProfiler, visualize)

In [ ]:
dd_incomes = dd.from_pandas(f_incomes.set_index('account_id'),
                            npartitions=10)
dd_dropouts = dd.from_pandas(dropouts, 
                             npartitions=5)

dd_merged = dd_dropouts.merge(dd_incomes)

dd_merged = dd_merged[
    (dd_merged.valid_from <= dd_merged.dropout_date) & 
    (dd_merged.dropout_date < dd_merged.valid_to)
]

dd_merged = dd_merged.groupby(
    ['account_id', 'dropout_date', 'dropout_motive']
).income.sum()

In [ ]:
dd_merged.visualize()

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.01) as rprof:
    final = dd_merged.compute()

visualize([prof, rprof], save=False);

In [ ]:
final = final.reset_index()
final = final.set_index(['account_id', 'dropout_date'])
merged = merged.set_index(['account_id', 'dropout_date'])
final.equals(merged)

### Thanks! Questions?


<div style="float: left;"><img src="imgs/man-qmark.jpg" width="300" align="middle"></div> 

<div>
<div>
  <img src="imgs/gmail-1162901_960_720.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">arielrossanigo@gmail.com</span>
</div>
<div>
  <img src="imgs/twitter-312464_960_720.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">@arielrossanigo</span>
</div>
<div>
  <img src="imgs/github-154769__340.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">https://github.com/arielrossanigo</span>
</div>
<div>
  <img src="imgs/Linkedin_icon.svg" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">https://www.linkedin.com/in/arielrossanigo/</span>
</div>

</div>

